# HSR Mujoco Example

HSR in Mujoco simulator.

### Dependencies:

1. [Giskard branch: mujoco_actions_devel](https://github.com/SemRoCo/giskardpy/tree/37b4048c3787c1f5f0370ca4e8ee10754eaa12c9)

1. [mujoco_sim](https://github.com/HoangGiang93/mujoco_sim)

1. [HSR description files](https://github.com/maltehue/mujoco_robots)

## 1. Launch Robot and envrionment

# What is the bootstrapping approach for the task learning?
In the context of teaching, the bootstrapping approach often involves starting with basic knowledge or skills and then iteratively improving or expanding upon them through additional learning experiences, in this case by interacting with a human mentor. This approach allows learners to gradually acquire more complex knowledge or abilities by building on what they already know. Bootstrapping is a common strategy in education and learning systems, helping individuals incrementally develop their expertise.

For the learning of the pouring task, a task-learning strategy has been developed by leveraging primitive actions that the PR2 robot can execute using GISKARD's primitive motion skills. This strategy involves breaking down the pouring task into already existing skills that the robot knows how to execute. By utilizing these primitive skills, the robot can gradually learn and perform the complete pouring task. When the instruction to save the learning plan is given, the robot can then incorporate this complex plan into its existing repertoire of skills. This method allows the robot to accumulate and store knowledge about various tasks and their constraints, contributing to its overall capabilities.

In [ ]:
import ipywidgets as widgets
from ipywidgets import HBox

# all available parameters
actions = [('pour', "tax:pour"),
           ('sprinkle', "tax:sprinkle")]

substances = [('water', "tax:water"),
              ('salt', "tax:salt"),
              ('pancake_batter', "tax:pancake_batter")]

sources = [('blue_cup_1', "tax:blue_cup_1"),
           ('white_cup_1', "tax:white_cup_1"),
           ('bowl_1', "tax:bowl_1"),
           ('pot_1', "tax:pot_1")]

destinations = [('blue_cup_2', "tax:blue_cup_2"),
                ('white_cup_2', "tax:white_cup_2"),
                ('bowl_2', "tax:bowl_2"),
                ('pot_2', "tax:pot_2")]

action = ""
substance = ""
source = ""
destination = ""


# Takes an Action of the list.
# Then makes the previously initialized variable global and assigns value of the given action to it 
def chooseAction(Action):
    global action
    action = Action


# Takes a Substance of the list.
# Then makes the previously initialized variable global and assigns value of the given substance to it 
def choosePref(Substance):
    global substance
    substance = Substance


# Takes a Source of the list.
# Then makes the previously initialized variable global and assigns value of the given source to it 
def chooseSource(Source):
    global source
    source = Source


# Takes a Destination of the list.
# Then makes the previously initialized variable global and assigns value of the given destination to it 
def chooseDestination(Destination):
    global destination
    destination = Destination


# Create the dropdown widgets
action_widget = widgets.Dropdown(options=actions, description='Action:')
substance_widget = widgets.Dropdown(options=substances, description='Substance:')
source_widget = widgets.Dropdown(options=sources, description='Source:')
destination_widget = widgets.Dropdown(options=destinations, description='Destination:')

widgets_display = HBox([action_widget, substance_widget, source_widget, destination_widget])
# Display widgets
widgets_display


### Initialize and display environment

In [1]:
import os
import psutil
import subprocess
import rospy
import ipywidgets as widgets
from ipywidgets import Button, Layout, GridBox, VBox, HBox, FloatSlider, ToggleButtons, Box
from IPython.display import display, Markdown, IFrame
from sidecar import Sidecar

# Directory of the ROS launch files
LAUNCH_FILE_DIR = os.path.abspath(os.path.join(os.getcwd(), "../launch"))
# To display web UI on the left
SIDECAR = None
# To manage the roslaunch process in the background
LAUNCH_PROCESS = None


# print("value of selected inputs", action, substance, source, destination)

# Popup the xpra desktop on the left panel
def open_xpra():
    global SIDECAR
    if SIDECAR is not None:
        SIDECAR.close()
    SIDECAR = Sidecar(title='XRPA', anchor='right')
    try:
        JUPYTERHUB_USER = '/user/' + os.environ['JUPYTERHUB_USER']
    except KeyError:
        JUPYTERHUB_USER = ''
    xpra_url = f"{JUPYTERHUB_USER}/xprahtml5/index.html"
    print(xpra_url)
    with SIDECAR:
        display(IFrame(src=xpra_url, width='100%', height='100%'))


try:
    DISPLAY_NUM = os.environ['DISPLAY']
except KeyError:
    DISPLAY_NUM = None

global LAUNCH_PROCESS
if LAUNCH_PROCESS is not None:
    LAUNCH_PROCESS.terminate()
    LAUNCH_PROCESS.wait()
if DISPLAY_NUM == ':100' or True:
    open_xpra()
launchfile = os.path.join(LAUNCH_FILE_DIR, "hsrb4s_velocity_two_cups.launch")
command = [
    'roslaunch',
    launchfile
]
LAUNCH_PROCESS = psutil.Popen(command,
                              stdout=subprocess.DEVNULL,
                              stderr=subprocess.DEVNULL)
print(f"""
    Executing command: 
    {' '.join(command)}
    in the background, the output will be hidden. 
    To check the output please execute it in a Terminial!
""")

launchfile = os.path.join(LAUNCH_FILE_DIR, "giskardpy_hsr_mujoco.launch")
command = [
    'roslaunch',
    launchfile
]
LAUNCH_PROCESS = psutil.Popen(command,
                              stdout=subprocess.DEVNULL,
                              stderr=subprocess.DEVNULL)
print(f"""
    Executing command: 
    {' '.join(command)}
    in the background, the output will be hidden. 
    To check the output please execute it in a Terminial!
""")



## 2. Control the robot with the Giskard Python Interface

In [1]:
import rospy
import random
from giskardpy.python_interface.python_interface import GiskardWrapper
from giskardpy.goals.joint_goals import JointPositionList
from giskardpy.monitors.joint_monitors import JointGoalReached
from geometry_msgs.msg import PoseStamped, Point, Quaternion, Vector3Stamped, PointStamped, QuaternionStamped
from grasp_and_pour_methods import openGripper, align_to, closeGripper, tilt, grasp, put_down, pick_up, attach_to_gripper, hsrGripperModel, pr2LeftGripperModel, ObjectGripperModel

# Before running this script make sure to start a giskard instance using 'roslaunch giskardpy giskardpy_hsr_mujoco.launch'
# And before that the mujoco simulation has to be running
rospy.init_node('graspAndPour')
gis = GiskardWrapper()
# choose a gripperModel for the gripper to be used in the following methods
gripper = hsrGripperModel

# Grasp the free cup from front side
pick_up(gis, gripper, 'free_cup', 'front', distance=0.04)

# Here the free_cup is added to the kinematic model of the robot.
# It is done to create a gripper model for the grasped objects to describe movements with respect to the cup
# TODO: Query the object size from environment information for now it is fixed
grasped_object_name = attach_to_gripper(gis, gripper, object_name='free_cup', object_size=(0.07, 0.07, 0.18))

# Create a new gripper model with the grasped and attached cup
cup_gripper = ObjectGripperModel(grasped_object_name=grasped_object_name)

# This aligns gripper to the left of the object frame in a distance of 0.13m.
# Additionally, the gripper frame is 0.2m higher than the object frame. The second_distance paramter can be used to
# to set an offset in the remaining dimension, here the x-axis of the object frame
align_to(gis, cup_gripper, 'left', object_frame='free_cup2', distance=0.10, height_offset=0.15, second_distance=0.0)

# Tilt the controlled_frame by angle around the rotation axis with a maximum velocity of velocity.
tilt(gis, cup_gripper, angle=1.7, velocity=1.0)

# TODO: wait till all the particles are outside the source, check the state of simulator?

# Tilt the controlled_frame by angle around the rotation axis with a maximum velocity of velocity.
tilt(gis, cup_gripper, angle=-1.7, velocity=1.0)

# TODO: putdown the cup again
# pos = "2 -0.6 0.5"
dest_cup_pose = PoseStamped()
dest_cup_pose.header.frame_id = 'map'
dest_cup_pose.pose.position = Point(2, -0.4, 0.5)
dest_cup_pose.pose.orientation.w = 1

put_down(gis, cup_gripper, dest_cup_pose)

AttributeError: 'ObjectGripperModel' object has no attribute 'effort_threshold'

Knowrob queries for accessing NEEM, roslaunch the environment before running the neem as animation

In [ ]:
assertz(collection_name(ros_tf)),
assertz(collection_name(triples)),
assertz(collection_name(annotations)),
assertz(collection_name(inferred)).

In [ ]:
collection_name(Name),
mng_get_db(DB, Collection, Name),
path_concat('/neem_data', Collection, Dir0),
mng_restore(DB, Dir0).

Get the information about the episode and the start and end time stamps for this episode

In [ ]:
is_episode(E), is_setting_for(E, A), is_action(A), has_time_interval(A, T), has_interval_begin(T,
                                                                                               Start), has_interval_end(
    T, End)

In [ ]:
tf_plugin: tf_republish_set_goal(Start, End)